# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [111]:
import numpy as np

from sklearn import datasets, cross_validation, tree, metrics, linear_model
from sklearn.datasets import load_boston
from sklearn.tree import DecisionTreeRegressor
#from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
import matplotlib
from matplotlib import pyplot as plt
 
boston = load_boston()
X, y = (boston.data, boston.target)
print(boston.data.shape)


(506, 13)


In [112]:
#X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.25, random_state=42)

In [113]:
train_len = int(X.shape[0]*0.75)
X_train=X[:train_len]
#X_train = X[:train_len, :]#same as above
X_test = X[train_len:]
y_train = y[:train_len]
y_test = y[train_len:]
#print('X_train shape', X_train.shape)
#print('X_test shape', X_test.shape)
#print('y_train shape', y_train.shape)

In [114]:
def file_answer(filename, str):
    file = open(filename, 'w')
    file.write(str)
    file.close()


## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [115]:
base_algorithms_list = [] #список с базовыми алгоритмами
coefficients_list = [] #список с коэффициентами перед алгоритмами
#algo = DecisionTreeRegressor(random_state=0)

In [116]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, 
            coefficients_list)])for x in X]

In [117]:
from sklearn.metrics import mean_squared_error
import math

In [118]:
mean_sq_err=[]#MSE - среднеквадратичная ошибка
gbm_predict_lst=[]
eta = 0.9
#начальный алгоритм

In [106]:

for i in range(50):
        coefficients_list.append(0.9/(1.0 + i))
        s = y_train - np.array(gbm_predict(X_train)) #y_tr replace by s here and below in fit
        base_algorithms_list.append(DecisionTreeRegressor(max_depth = 50, random_state = 42).fit(X_train,s))
print (pow(mean_squared_error(y_test, gbm_predict(X_test)),0.5))

5.72260267314


In [60]:
file_answer('2.txt', str(5.72260267314))

In [120]:
base_algorithms_list = [] #список с базовыми алгоритмами
coefficients_list = [] #список с коэффициентами перед алгоритмами
mean_sq_err=[]
gbm_predict_lst=[]
eta = 0.9/(1.0 + 0)

In [121]:
#начальный алгоритм
algo_init = tree.DecisionTreeRegressor(max_depth=5 , random_state=42)
algo_init.fit(X_train, y_train)
base_algorithms_list.append(algo_init)
coefficients_list.append(eta)
s=0

In [122]:
for k in range(50):
    s = y_train-gbm_predict(X_train)
    clf = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    clf.fit(X_train, s)
    base_algorithms_list.append(clf)
    coefficients_list.append((eta/(1.0 + (k+1))))
    #m2 = (metrics.mean_squared_error(y_test, gbm_predict(X_test)))**0.5
    m2=pow(mean_squared_error(y_test, gbm_predict(X_test)),0.5)
    mean_sq_err.append(m2)

print (pow(mean_squared_error(y_test, gbm_predict(X_test)),0.5))
#print(m2)

4.8101206862


In [123]:
file_answer('3.txt', str(4.8101206862))

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [124]:
# Test number of trees increased by 200
base_algorithms_list = [] #список с базовыми алгоритмами
coefficients_list = [] #список с коэффициентами перед алгоритмами
mean_sq_err=[]
gbm_predict_lst=[]
eta = 0.9/(1.0 + 0)
#начальный алгоритм
algo_init = tree.DecisionTreeRegressor(max_depth=5 , random_state=42)
algo_init.fit(X_train, y_train)
base_algorithms_list.append(algo_init)
coefficients_list.append(eta)
s=0
for k in range(200):
    s = y_train-gbm_predict(X_train)
    clf = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    clf.fit(X_train, s)
    base_algorithms_list.append(clf)
    coefficients_list.append((eta/(1.0 + (k+1))))
    m2=pow(mean_squared_error(y_test, gbm_predict(X_test)),0.5)
    mean_sq_err.append(m2)

print (pow(mean_squared_error(y_test, gbm_predict(X_test)),0.5))


4.80791197817


2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.
MSE=4.8101206862 при 50 деревьях . При 200 деревьях  MSE=4.80791197817 Just a little impovement but the time for calculation substantianlly increased

In [125]:
# Test 2- max_depth increased by 15
base_algorithms_list = [] #список с базовыми алгоритмами
coefficients_list = [] #список с коэффициентами перед алгоритмами
mean_sq_err=[]
gbm_predict_lst=[]
eta = 0.9/(1.0 + 0)
#начальный алгоритм
algo_init = tree.DecisionTreeRegressor(max_depth=15 , random_state=42)
algo_init.fit(X_train, y_train)
base_algorithms_list.append(algo_init)
coefficients_list.append(eta)
s=0
for k in range(50):
    s = y_train-gbm_predict(X_train)
    clf = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    clf.fit(X_train, s)
    base_algorithms_list.append(clf)
    coefficients_list.append((eta/(1.0 + (k+1))))
    m2=pow(mean_squared_error(y_test, gbm_predict(X_test)),0.5)
    mean_sq_err.append(m2)

print (pow(mean_squared_error(y_test, gbm_predict(X_test)),0.5))

6.25592312417


max_depth= 5 MSE=4.8101206862 and max_depth=15 MSE=6.25592312417 
С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.


In [127]:
file_answer('4.txt', '2 3')

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [126]:
linear = linear_model.LinearRegression()
linear.fit(X_train, y_train)
linear_error = metrics.mean_squared_error(y_test, linear.predict(X_test))**0.5
print("Linear regression test results: ", linear_error)
file_answer('5.txt', str(linear_error))

Linear regression test results:  8.27046803494


In [ ]:
file_answer('5.txt', str(7.84812179648))